# Basic Initialization 

In [1]:
import os
import cv2
import numpy as np

import imagehash
from PIL import Image
from rembg import remove
import imghdr
import dlib

from tqdm import tqdm

import io
import time
from shutil import move
import io
from IPython.display import clear_output

from tqdm import tqdm

import re
import pandas as pd
from collections import Counter

# Basic Image Processing

### Basic Init for Image Helper

In [78]:
# -------------------- [Enter the Location of the folder w/files inside it] -------------------
folder_loc = r""

# -------------------- [For Duplicate Image Removal] ------------------------------------------
# This is used to remove images that are much smaller and will only artefact if you use them
# These values: 700, 700, are suggested, you can change it to any "SQUARE" size that you want
# Square => x = y
x_thres = 700
y_thres = 700

# -------------------- [For Suitability Check] ------------------------------------------------

# Format for below: Abbreviation_full_form(init value)
# Min_Confidence(0.9), Min_Size(0.009) (proportional to image size), Min_SharpeningStat(100), Blur_Threshold(100)
mc, ms, mss, bt = 0.9, 0.01, 100, 100

# The best <x> images will be moved to the folder, if you want all, leave it at 0
top_n_img_count = 0

# -------------------- [For face crop] --------------------------------------------------------

# Face Crop to dimensions x, y; eg(512, 512)
# IMP: Keep it square to prevent any code from breaking, thanks
x_crop = 512
y_crop = 512

# -------------------- [Do not touch this] ----------------------------------------------------
detector = dlib.get_frontal_face_detector()
selected_folder = os.path.join(folder_loc, "SelectedImages")

### Convert all images to PNG for easier reading [Heavily Suggested]

In [ ]:
def convert_images_to_png(source_folder):
    if not os.path.exists(source_folder):
        print(f"The folder {source_folder} does not exist.")
        return
    for filename in tqdm(os.listdir(source_folder), desc = "Processing images to .png"):
        if filename.lower().endswith(('.jpg', '.jpeg', '.bmp', '.gif', '.tiff', '.webp')):
            file_path = os.path.join(source_folder, filename)
            with Image.open(file_path) as img:
                new_filename = os.path.splitext(filename)[0] + '.png'
                new_file_path = os.path.join(source_folder, new_filename)
                try:
                    img.save(new_file_path, 'PNG')
                    # os.remove(file_path)
                except:
                    img.convert('RGB').save(new_file_path, "PNG", optimize=True)
            os.remove(file_path)

convert_images_to_png(folder_loc)

### Remove Same Images

In [ ]:
def remove_black_bars(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest_contour)
        cropped_image = image[y:y+h, x:x+w]
        return cropped_image
    return image

def process_images(folder_path, x, y):
    duplicate_images_folder = os.path.join(folder_path, "Duplicate_Images")
    if not os.path.exists(duplicate_images_folder):
        os.makedirs(duplicate_images_folder)

    small_images_folder = os.path.join(folder_path, "Small_Images")
    if not os.path.exists(small_images_folder):
        os.makedirs(small_images_folder)
    
    image_hashes = {}
    files = [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp'))]
    for filename in tqdm(files, desc="Processing Images"):
        file_path = os.path.join(folder_path, filename)
        image = cv2.imread(file_path)
        if image is None:
            continue
        
        image_no_black_bars = remove_black_bars(image)
        
        pil_image = Image.fromarray(cv2.cvtColor(image_no_black_bars, cv2.COLOR_BGR2RGB))
        h,w = pil_image.size
        # print(filename, "\n",h*w, x*y)            
        
        hash = str(imagehash.average_hash(pil_image))
        
        # Check for duplicates
        if hash in image_hashes:
            print(f"Duplicate found: {filename} is a duplicate of {image_hashes[hash]}")
            os.rename(file_path, os.path.join(duplicate_images_folder, filename))
        else:
            image_hashes[hash] = filename
            if h*w < x*y:
                os.rename(file_path, os.path.join(small_images_folder, filename))
    
    if len([i for i in os.listdir(duplicate_images_folder)]) == 0:
        os.rmdir(duplicate_images_folder)

process_images(folder_loc, x_thres, y_thres)

### Checks Images for Suitability Check [Now with Top "x" Images option]

In [ ]:
current_folder = os.getcwd()

modelFile = os.path.join(current_folder, "opencv_face_detector_uint8.pb")
configFile = os.path.join(current_folder, "opencv_face_detector.pbtxt")

if os.path.exists(modelFile) != True or os.path.exists(configFile) != True:
    print("ERROR: KEY FILES FOR RUNNING THIS MODEL NOT FOUND, PLEASE FIND THEM AND INSTALL THEM")

else:
    net = cv2.dnn.readNetFromTensorflow(modelFile, configFile)

    def calculate_image_sharpness(image):
        
        """
        Calculate the sharpness of an image using the variance of the Laplacian
        """
        
        if image is None or image.size == 0:
            return 0
        return cv2.Laplacian(image, cv2.CV_64F).var()

    def is_image_blurry(image, blur_threshold):
        
        """
        Check if an image is blurry using the variance of the Laplacian method.
        """
        
        variance_of_laplacian = cv2.Laplacian(image, cv2.CV_64F).var()
        return variance_of_laplacian < blur_threshold

    def detect_faces_and_evaluate(image, min_confidence, min_size, min_sharpness, blur_threshold):
        
        """
        Detect faces in an image and evaluate based on size, sharpness, and blurriness.
        """
        
        if image is None or image.size == 0:
            return []
        if is_image_blurry(image, blur_threshold):
            return []
        (h, w) = image.shape[:2]
        blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()
        faces_detected = []
        for i in range(0, detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > min_confidence:
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")
                startX, startY, endX, endY = max(0, startX), max(0, startY), min(w, endX), min(h, endY)
                if startX >= endX or startY >= endY:
                    continue
                face_region = image[startY:endY, startX:endX]
                if face_region.size == 0:
                    continue
                face_size = (endX - startX) * (endY - startY)
                face_sharpness = calculate_image_sharpness(face_region)
                if face_size > min_size and face_sharpness > min_sharpness:
                    faces_detected.append((confidence, face_size, face_sharpness))
        return faces_detected

    def remove_black_bars(image):
        
        """
        Remove black bars from an image.
        """

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if contours:
            largest_contour = max(contours, key=cv2.contourArea)
            x, y, w, h = cv2.boundingRect(largest_contour)
            cropped_image = image[y:y+h, x:x+w]
            return cropped_image
        return image

def select_best_images(
        folder_path:str, 
        min_confidence:float = 0.9, 
        min_size:float = 0.009, 
        min_sharpness:float = 100, 
        blur_threshold:float = 100,
        top_n:int = 0
        ):
    
    selected_images_folder = os.path.join(folder_path, "SelectedImages")
    if not os.path.exists(selected_images_folder):
        os.makedirs(selected_images_folder)
    
    image_ratings = []  # List to store ratings and file paths
    
    files = [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp'))]
    for filename in tqdm(files, desc="Evaluating Images"):
        file_path = os.path.join(folder_path, filename)
        image = cv2.imread(file_path)
        if image is None:
            continue
        
        image_no_black_bars = remove_black_bars(image)
        
        faces_detected = detect_faces_and_evaluate(image_no_black_bars, min_confidence, min_size * image_no_black_bars.size, min_sharpness, blur_threshold)
        if faces_detected:
            rating = sum([face[0] + face[1] + face[2] for face in faces_detected]) 
            image_ratings.append((rating, file_path))
    
    # Sort images based on ratings
    if top_n == 0:
        top_n = len(files)
    top_images = sorted(image_ratings, key=lambda x: x[0], reverse=True)[:top_n]
    
    for _, top_image_path in top_images:
        filename = os.path.basename(top_image_path)
        move(top_image_path, os.path.join(selected_images_folder, filename))



select_best_images(folder_loc, mc, ms, mss, bt, top_n_img_count)

selected_folder = os.path.join(folder_loc, "SelectedImages")

## Advanved Image Cropping

In [79]:
if os.path.exists(selected_folder) != True:
    selected_folder = folder_loc

### Remove Background of Images

In [ ]:
def remove_background_from_images(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    for filename in tqdm(image_files, desc="Processing Images"):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)

        with open(input_path, 'rb') as input_file:
            input_image = input_file.read()

            output_image = remove(input_image)

            output_image = Image.open(io.BytesIO(output_image))
            output_image.save(output_path)

remove_background_from_images(selected_folder, os.path.join(selected_folder, "No_BG"))

### Crop Image to Face

In [ ]:
failed_img = []
face_failed = []

def faceCrop(folder_dir, fName, img, imp=1, x=512, y=512):
    detector = dlib.get_frontal_face_detector()
    _, ext = os.path.splitext(fName)
    
    fName = fName.split(".")[0]
    if ext.lower() not in ['.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp']:
        print(f"Unsupported image format: {ext}")
        failed_img.append(fName)
        return

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # getting orignal dimensions
    # print(len(gray), len(gray[0]))
    oh, ow = len(gray), len(gray[0])
    faces = detector(gray)
    # print(fName)
    # print("original dims:", ow, oh)
    if len(faces) == 0:
        face_failed.append(fName)
    else:
        areas = [face.width() * face.height() for face in faces]
        max_area_index = np.argmax(areas)
        fx, fy, fw, fh = faces[max_area_index].left(), faces[max_area_index].top(), faces[max_area_index].width(), faces[max_area_index].height()

        # Calculate center of the face
        cx, cy = fx + fw // 2, fy + fh // 2

        # print("old: ", fw, fh)
        fw, fh = int(fw * imp), int(fh * imp)
        # print("new: ", fw, fh)
        if min(fw, fh) > min(ow, oh):
            fw = int(min(ow, oh))
            fh = int(min(ow, oh))
            # print("fixed dims:", fw, fh)
        # fw, fh = min(ow, oh), min(ow, oh)
        # print("updated: ", fw, fh)

        # Determine the size of the square bounding box
        max_dim = max(fw, fh)
        half_dim = max_dim // 2

        # Calculate new top-left corner coordinates
        fx = cx - half_dim
        fy = cy - half_dim

        # Ensure the bounding box stays within image bounds
        fx = max(0, fx)
        fy = max(0, fy)
        # fw = min(img.shape[1] - fx, max_dim)
        # fh = min(img.shape[0] - fy, max_dim)
        fw = min(img.shape[1] - fx, img.shape[0] - fy)
        fh = min(img.shape[1] - fx, img.shape[0] - fy)

        # print("Remade dims:", fw, fh)

        # Crop the square bounding box
        cropped = img[fy:fy+fh, fx:fx+fw]

        # Resize the cropped image to the specified dimensions
        cropped = cv2.resize(cropped, (x, y))

        cv2.imwrite(os.path.join(folder_dir, fName + '.png'), cropped)

def main_call(folder_path, x=512, y=512, imp = 1):
    resized_folder_path = os.path.join(folder_path, "Cropped_Images")
    
    if not os.path.exists(resized_folder_path):
        print(r"Made /resized folder")
        os.makedirs(resized_folder_path)

    if os.path.isdir(folder_path):
        print("Valid Folder Location")
        files = os.listdir(folder_path)
        files_img = [i for i in files if not i.endswith('.txt') and not os.path.isdir(os.path.join(folder_path, i))]
        print("Number of files: ", len(files_img))
        itr = 0
        for i in tqdm(files_img, desc="Processing Files"):
            itr += 1
            file_loc = os.path.join(folder_path, i)
            if os.path.isfile(file_loc) and imghdr.what(file_loc):
                img = cv2.imread(file_loc)
                if img is None:
                    print(f"Failed to load image: {i}:", "Cause: Empty !")
                    failed_img.append(i)
                    continue
                else:
                    faceCrop(resized_folder_path, i, img, imp, x, y)
            elif os.path.isdir(file_loc):
                print(i, ": Is a folder")
            elif i.endswith('.txt'):
                None
            else:
                print(i, ": Is not a supported Image File")

    # clear_output(wait=True)

    time.sleep(3)

    print("Image Cropping Completed")

    time.sleep(2)

    if len(failed_img) != 0:
        print("\nThese images failed: \nReason: Invalid to load: \n", failed_img, "\nCount:", len(failed_img))

    if len(face_failed) != 0:
        print("\nThese images failed: \nReason: Face not found: \n", face_failed, "\nCount:", len(face_failed))

# --------------------> If didn't use remBG <--------------------
# main_call(selected_folder, x_crop, y_crop, imp = 2)

# --------------------> If you do use remBG <--------------------
# IMP is a variable that allows the cropping boundary to increase by img_size*imp, suggested: imp = 2
main_call(os.path.join(selected_folder, "No_BG"), x_crop, y_crop, imp = 2)

### [IN WORK] [DO NOT RUN THIS BLOCK] Crop X BEST Images to face, rest zoomed out body crop

In [ ]:
# def more_face_crop(folder_dir, fName, img, x=512, y=512):
#     detector = dlib.get_frontal_face_detector()
#     _, ext = os.path.splitext(fName)
    
#     fName = fName.split(".")[0]
#     if ext.lower() not in ['.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp']:
#         print(f"Unsupported image format: {ext}")
#         failed_img.append(fName)
#         return
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     faces = detector(gray)
#     print(faces)
#     if len(faces) == 0:
#         None
#         # print("--> TO BE ADDED")
#         face_failed.append(fName)
#     else:
#         # Calculate the maximum bounding box around all detected faces
#         fx_min, fy_min, fw_max, fh_max = float('inf'), float('inf'), x, y
#         for face in faces:
#             fx, fy, fw, fh = face.left(), face.top(), face.width(), face.height()
#             print(fx_min, fy_min, fw_max, fh_max)
#             print(fx, fy, fw, fh)

#             cv2.rectangle(img, (fx, fy), (fx + fw, fy + fh), (255, 0, 0), 2)

#             cropped_p = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
#             cropped_p.show()

#             fx_min = min(fx_min, fx)
#             fy_min = min(fy_min, fy)
#             fw_max = max(fw_max, fx + fw)
#             fh_max = max(fh_max, fy + fh)

#             cv2.rectangle(img, (fx, fy), (fx + fw, fy + fh), (0, 255, 0), 2)

#             cropped_p = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
#             cropped_p.show()

#         # Create a square bounding box with top aligned to the topmost area of the head
#         side_length = max(fw_max - fx_min, fh_max - fy_min)
#         cx = (fx_min + fw_max) // 2
#         cy = fy_min + 10  # Add a 10-pixel gap
#         fx = cx - side_length // 2
#         fy = cy - side_length // 2
#         fw = side_length
#         fh = side_length

#         # Crop the image to the square bounding box
#         # cropped = img[fy:fy+fh, fx:fx+fw]
#         cv2.rectangle(img, (fx, fy), (fx + fw, fy + fh), (0, 0, 255), 2)

#         # Resize the cropped image to the specified dimensions
#         # cropped = cv2.resize(cropped, (x, y))
#         # cv2.imshow("img", cropped)
#         cropped_p = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
#         cropped_p.show()
                                    

#         # cv2.imwrite(os.path.join(folder_dir, fName + '.png'), cropped)
    
# def main_callX(folder_loc, x, y):
#     file_loc = r"C:\Users\parvs\Downloads\Alia\SelectedImages\SelectedImages\605804587387098720.png"
#     fName = r"605804587387098720.png"
#     if os.path.isfile(file_loc) and imghdr.what(file_loc):
#             img = cv2.imread(file_loc)
#             if img is None:
#                 print(f"Failed to load image:", "Cause: Empty !")
#                 # failed_img.append(i)
#             more_face_crop(folder_loc, fName, img, x, y)

# # Example usage:
# main_callX(r"C:\Users\parvs\Downloads\Alia\SelectedImages\SelectedImages", 512, 512)

# Image Captioning Helper

## Basic Imports

In [84]:
# Location of folder along with caption files in a .txt format
folder_loc = r''

# Maximum count of words to show in "most used words"
max_word_count = 100

# Unique name of character
charName = ''

## Display most USED words in captions

In [ ]:
def get_most_common_words(folder_path, mc=max_word_count):
    
    mc = int(mc)
    
    word_counts = Counter()
    word_order = []
    
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                words = file.read().split(", ")
                for word in words:
                    if word not in word_order:
                        word_order.append(word)
                word_counts.update(words)
    
    common_words_df = pd.DataFrame(word_counts.most_common(mc), columns=['Word', 'Frequency'])
    common_words_dict = dict(word_counts.most_common(mc))
    
    chronological_list = sorted(common_words_dict.keys(), key=lambda word: common_words_dict[word], reverse=True)
    print("Common words: ", "\n", common_words_df, "\n", "Common words Dictionary: ", "\n", common_words_dict, "\n", "Common words List: ", "\n", chronological_list, "\n")
    
    return common_words_df, common_words_dict, chronological_list

df, word_dict, word_list = get_most_common_words(folder_loc)

## Select the words that you want to delete

In [94]:
# Add to this list
words_to_delete_list = ['photo_\\(medium\\)', '3d', 'blurry', 'blur']

## Update .txt files with required words (to be removed/added/updated)

In [ ]:
def process_files(folder_path, words_list=words_to_delete_list, charName=charName):
    for filename in tqdm(os.listdir(folder_path), desc="Processing Caption Files"):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.txt'):
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read().lower()
                words = content.split(", ")
                # print("\n", words)

            # Create a new list without the words to delete
            updated_words = [word for word in words if word not in words_list]

            # Add UniqueWord to the first of the list if not exist
            if words[0] != charName.lower() or words[0] != charName:
                # print(words[0].lower())
                updated_words.insert(0, charName)

            # Join the updated words back into a string
            updated_content = ', '.join(updated_words)
            # print("\nUpdated:", updated_content)

            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(updated_content)

process_files(folder_loc, words_to_delete_list, charName)

## Clear VRAM

In [ ]:
# PyTorch
import torch
torch.cuda.empty_cache()  # Releases all unoccupied cached memory currently held by the caching allocator

# CUDA (via Numba)
from numba import cuda
cuda.select_device(0)  # Selects the GPU of ID 0 (change according to your setup)
cuda.close()  # Closes the device, attempting to free all memory allocated